In [19]:
from sklearn import svm, datasets
iris = datasets.load_iris()

import pandas as pd
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])

df.head(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [20]:
from sklearn.model_selection import GridSearchCV

model_paramaters = {
    'C': [1, 10, 20],
    'kernel': ['rbf', 'linear']
}

model = GridSearchCV(svm.SVC(gamma='auto'), model_paramaters, cv=5, return_train_score=False)

model.fit(iris.data, iris.target)

df_results = pd.DataFrame(model.cv_results_)
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000546,4.458286e-04,0.000364,4.459455e-04,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000728,3.639698e-04,0.000000,0.000000e+00,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000182,3.638268e-04,0.000546,4.458675e-04,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000910,2.336015e-07,0.000000,0.000000e+00,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000546,4.458676e-04,0.000182,3.640175e-04,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000000,0.000000e+00,0.000910,6.843901e-07,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [21]:
df_results[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [22]:
model.best_params_

{'C': 1, 'kernel': 'rbf'}

In [23]:
Score = ((model.best_score_) * 100).round(2)
print(f'{Score}%')

98.0%


In [26]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

classification_models = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

scores = []

for model_choice, parameter in classification_models.items():
    model =  GridSearchCV(parameter['model'], parameter['params'], cv=5, return_train_score=False)
    model.fit(iris.data, iris.target)
    
    scores.append({
        'model': model_choice,
        'best_score': model.best_score_,
        'best_params': model.best_params_
    })
    
results = pd.DataFrame(scores,columns=['model','best_score','best_params'])
results['best_score'] = (results['best_score'] * 100).map('{:.2f}%'.format)
results.sort_values(by='best_score', ascending=False, inplace=True)

results

,model,best_score,best_params
0,svm,98.00%,"{'C': 1, 'kernel': 'rbf'}"
2,logistic_regression,96.67%,{'C': 5}
1,random_forest,96.00%,{'n_estimators': 10}
